In [1]:
# !pip install opencv-python scikit-learn scikit-image matplotlib spectral keras_tuner vis
# !pip install tensorflow numpy pandas
# !pip install spectral

In [2]:
import tensorflow as tf

2023-01-06 18:49:53.730307: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
from tensorflow import keras as keras
from keras import layers as layers

In [4]:
import os, timeit
from skimage.filters import threshold_otsu
import numpy as np
from math import inf as inf

In [5]:
from spectral.io import envi as envi
from spectral import imshow

In [6]:
from sklearn.decomposition import IncrementalPCA

In [7]:
import sys

In [8]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  0


2023-01-06 18:49:56.009891: E tensorflow/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2023-01-06 18:49:56.009918: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (root1-HP-Z8-G4-Workstation): /proc/driver/nvidia/version does not exist


In [9]:
from sys import platform
DATA_DIRECTORY = ""
SLASH = ""
if platform == "linux" or platform == "linux2":
    DATA_DIRECTORY = "/home/nitintyagi/wheat data/BULK/"
    SLASH = "/"
elif platform == "win32":
    DATA_DIRECTORY = "D:\mvl\wheat\data\BULK\\"
    SLASH="\\"

In [10]:
BAND_NUMBER = 60
FILLED_AREA_RATIO = 0.9
TOTAL_IMAGE_COUNT = 2400
IMAGE_COUNT = int(TOTAL_IMAGE_COUNT/4)
NUM_VARIETIES = 4

IMAGE_WIDTH = 30
IMAGE_HEIGHT = 30

In [11]:
ACTIVATION_TYPE =  "relu"
BATCH_SIZE = 2*NUM_VARIETIES

LEARNING_RATE_BASE = 0.0001

In [12]:
from enum import Enum
class filter_method(Enum):
    none = 0
    snv = 1
    msc = 2
    
FILTER = filter_method(2).name

In [13]:
from enum import Enum
 
class feature_extraction_method(Enum):
    none = 0
    pca_loading = 1
    lda = 2
    ipca = 3

FEATURE_EXTRACTION = feature_extraction_method(0).name

NUM_OF_BANDS = 3
if FEATURE_EXTRACTION == "pca_loading" or FEATURE_EXTRACTION == "ipca":
    NUM_OF_BANDS = 8
elif FEATURE_EXTRACTION == "lda":
    NUM_OF_BANDS = 3
    assert NUM_OF_BANDS <= min(NUM_VARIETIES-1,168),"NUM_OF_BANDS is greater."


REMOVE_NOISY_BANDS = False
FIRST_BAND = 21
LAST_BAND = 149

In [14]:
def start_timer():
    print("Testing started")
    return timeit.default_timer()

def end_timer():
    return timeit.default_timer()

def show_time(tic,toc): 
    test_time = toc - tic
    print('Testing time (s) = ' + str(test_time) + '\n')

In [15]:
# List for All varieties
VARIETIES = []
VARIETIES_CODE = {}

for name in os.listdir(DATA_DIRECTORY):
    if (name.endswith(".hdr") or name.endswith(".bil")):
        continue
    VARIETIES_CODE[name] = len(VARIETIES)
    VARIETIES.append(name)
    if len(VARIETIES)==NUM_VARIETIES:
        break

In [16]:
def dataset_file_name(variety):
    return "./dataset/V"+str(variety).zfill(3)+"_IC_"+str(TOTAL_IMAGE_COUNT).zfill(5)+"_FilledArea_"+str(FILLED_AREA_RATIO)+"_NumOfBands_"+str(NUM_OF_BANDS)+"_FB_"+str(FIRST_BAND)+"_LB_"+str(LAST_BAND)+"_BandNo_"+str(BAND_NUMBER)+"_ImageHeight_"+str(IMAGE_HEIGHT)+"_ImageWidth_"+str(IMAGE_WIDTH)+"_FILTER_"+str(FILTER)+"_FeatureExtraction_"+str(FEATURE_EXTRACTION)

In [17]:
train_dataset = []
train_dataset_label = []
test_dataset=[]
test_dataset_label = []

for idx, v in enumerate(VARIETIES):
    print("idx: ",idx)
    if idx >= NUM_VARIETIES:
        break
    train_dataset= train_dataset + np.load(dataset_file_name(v)+"_train_dataset.npy").tolist()
    train_dataset_label = train_dataset_label + np.load(dataset_file_name(v)+"_train_dataset_label.npy").tolist()
    test_dataset = test_dataset + np.load(dataset_file_name(v)+"_test_dataset.npy").tolist()
    test_dataset_label = test_dataset_label + np.load(dataset_file_name(v)+"_test_dataset_label.npy").tolist()

train_dataset = np.array(train_dataset)
train_dataset_label = np.array(train_dataset_label)
test_dataset = np.array(test_dataset)
test_dataset_label = np.array(test_dataset_label)

idx:  0
idx:  1
idx:  2
idx:  3


In [18]:
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras import datasets, layers, models, losses, Model
from keras.layers import Input, Conv2D, MaxPool2D, MaxPooling2D, Activation, Flatten, Dense, AveragePooling2D, GlobalAveragePooling2D
from keras.layers.core import Dropout
from keras.optimizers import Adam

In [19]:
def normalizeDataWholeSeed(data,normalization_type='max'):
    
    if normalization_type == 'max':
        for idx in range(data.shape[0]):
            data[idx,:,:,:] = data[idx,:,:,:]/np.max(abs(data[idx,:,:,:]))
            
    elif normalization_type == 'l2norm':
        from numpy import linalg as LA
        for idx in range(data.shape[0]):
            data[idx,:,:,:] = data[idx,:,:,:]/LA.norm(data[idx,:,:,:])       
        
    return data

In [20]:
def inception(x,filters_1x1,filters_3x3_reduce,filters_3x3,filters_5x5_reduce,filters_5x5,filters_pool,activation_type='relu'):
    path1 = Conv2D(filters_1x1,        (1, 1), padding='same', activation=activation_type)(x)
    
    path2 = Conv2D(filters_3x3_reduce, (1, 1), padding='same', activation=activation_type)(x)
    path2 = Conv2D(filters_3x3,        (1, 1), padding='same', activation=activation_type)(path2)
    
    path3 = Conv2D(filters_5x5_reduce, (1, 1), padding='same', activation=activation_type)(x)
    path3 = Conv2D(filters_5x5,        (1, 1), padding='same', activation=activation_type)(path3)
    
    path4 = MaxPool2D((3, 3),  strides=(1, 1), padding='same')(x)
    path4 = Conv2D(filters_pool,       (1, 1), padding='same', activation=activation_type)(path4)
    
    return tf.concat([path1, path2, path3, path4], axis=3)

In [21]:
def auxiliary_classifier(x,num_classes,activation_type='relu'):
    aux = AveragePooling2D((5, 5), strides=3)(x)
    aux = Conv2D(128, 1, padding='same', activation=activation_type)(aux)
    aux = Flatten()(aux)
    aux = Dense(1024, activation=activation_type)(aux)
    aux = Dropout(0.7)(aux)
    aux = Dense(num_classes, activation='softmax')(aux)
    return aux

In [22]:
def GoogleNetModel(data_num_rows, data_num_cols, num_input_chans=1, num_classes=NUM_VARIETIES, activation_type='relu', dropout_rate=0.0):

    inp = Input(shape=(data_num_rows, data_num_cols, num_input_chans))
    input_tensor = layers.experimental.preprocessing.Resizing(224, 224, interpolation="bilinear", input_shape=train_dataset.shape[1:])(inp)
    x = Conv2D(64,  7, strides=2, padding='same', activation=activation_type)(input_tensor)
    x = MaxPooling2D(3, strides=2)(x)
    x = Conv2D(64,  1, strides=1, padding='same', activation=activation_type)(x)
    x = Conv2D(192, 3, strides=1, padding='same', activation=activation_type)(x)
    x = MaxPooling2D(3, strides=2)(x)
    x = inception(x, filters_1x1=64 , filters_3x3_reduce=96 , filters_3x3=128, filters_5x5_reduce=16, filters_5x5=32, filters_pool=32)
    x = inception(x, filters_1x1=128, filters_3x3_reduce=128, filters_3x3=192, filters_5x5_reduce=32, filters_5x5=96, filters_pool=64)
    x = MaxPooling2D(3, strides=2)(x)
    x = inception(x, filters_1x1=192, filters_3x3_reduce=96 , filters_3x3=208, filters_5x5_reduce=16, filters_5x5=48, filters_pool=64)
    
    aux1 = auxiliary_classifier(x,num_classes)
    
    x = inception(x, filters_1x1=160, filters_3x3_reduce=112, filters_3x3=224, filters_5x5_reduce=24, filters_5x5=64, filters_pool=64)
    x = inception(x, filters_1x1=128, filters_3x3_reduce=128, filters_3x3=256, filters_5x5_reduce=24, filters_5x5=64, filters_pool=64)
    x = inception(x, filters_1x1=112, filters_3x3_reduce=144, filters_3x3=288, filters_5x5_reduce=32, filters_5x5=64, filters_pool=64)
    
    aux2 = auxiliary_classifier(x,num_classes)
    
    x = inception(x, filters_1x1=256, filters_3x3_reduce=160, filters_3x3=320, filters_5x5_reduce=32, filters_5x5=128, filters_pool=128)
    x = MaxPooling2D(3, strides=2)(x)
    x = inception(x, filters_1x1=256, filters_3x3_reduce=160, filters_3x3=320, filters_5x5_reduce=32, filters_5x5=128, filters_pool=128)
    x = inception(x, filters_1x1=384, filters_3x3_reduce=192, filters_3x3=384, filters_5x5_reduce=48, filters_5x5=128, filters_pool=128)
    x = GlobalAveragePooling2D()(x)
    x = Dropout(dropout_rate)(x)
    out = Dense(num_classes, activation='softmax')(x)
    
    return Model(inputs = inp, outputs = [out, aux1, aux2])

In [23]:
def getGoogleNetModel():
    learning_rate_base = LEARNING_RATE_BASE
    activation_type = ACTIVATION_TYPE
    wheat_types =  VARIETIES
    num_classes = len(wheat_types)
    dropout_rate = 0.4
    print("--------------Load Data--------------")

    x_training = np.array(train_dataset)
    labels_training = np.array(train_dataset_label)
    
    # Normalize the data
    x_training = normalizeDataWholeSeed(x_training)
    
    # Extract some information
    num_train = x_training.shape[0]
    N_spatial = x_training.shape[1:3]
    N_channel = x_training.shape[3]
    
    print("--------------Done--------------")
    
    ############ Create a model ############
    print("--------------Create a model--------------")
    
    # Generate a model
    model = GoogleNetModel(data_num_rows = N_spatial[0], 
                           data_num_cols = N_spatial[1],
                           num_input_chans = N_channel, 
                           num_classes = num_classes,
                           activation_type = activation_type,
                           dropout_rate = dropout_rate)

    # Compile the model
    adam_opt = Adam(learning_rate=LEARNING_RATE_BASE, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.01)
    model.compile(optimizer=adam_opt, loss=[losses.sparse_categorical_crossentropy,losses.sparse_categorical_crossentropy,losses.sparse_categorical_crossentropy],loss_weights=[1, 0.3, 0.3],metrics=['accuracy'])
    print("---------Completed---------")
    return model

In [24]:
model = getGoogleNetModel()

--------------Load Data--------------
--------------Done--------------
--------------Create a model--------------


2023-01-06 18:54:46.930351: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


---------Completed---------


In [25]:
x_train = []
y_train = []
x_val = []
y_val = []

for i in range(len(train_dataset)):
    if i%5==0:
        x_val.append(train_dataset[i])
        y_val.append(train_dataset_label[i])
    else:
        x_train.append(train_dataset[i])
        y_train.append(train_dataset_label[i])
        
x_train = np.array(x_train)
y_train = np.array(y_train)
y_train = [y_train,y_train,y_train]

x_val = np.array(x_val)
y_val = np.array(y_val)
y_val = [y_val,y_val,y_val]

In [26]:
for x in range(2):
    print("From epochs: ",20*x+1," to ",20+20*x)
    tic = start_timer()
    model.fit(x_train ,y_train ,batch_size=BATCH_SIZE ,epochs=20, verbose=2, validation_data=(x_val, y_val), shuffle=True)
    toc = end_timer()
    show_time(tic,toc)
    print("for testing")
    print(model.evaluate(test_dataset,test_dataset_label))
    print("for training")
    print(model.evaluate(train_dataset,train_dataset_label))

From epochs:  1  to  20
Testing started
Epoch 1/20
768/768 - 1410s - loss: 2.1191 - dense_4_loss: 1.3200 - dense_1_loss: 1.3307 - dense_3_loss: 1.3330 - dense_4_accuracy: 0.3387 - dense_1_accuracy: 0.3366 - dense_3_accuracy: 0.3237 - val_loss: 2.0042 - val_dense_4_loss: 1.2488 - val_dense_1_loss: 1.2571 - val_dense_3_loss: 1.2610 - val_dense_4_accuracy: 0.4108 - val_dense_1_accuracy: 0.3984 - val_dense_3_accuracy: 0.3997 - 1410s/epoch - 2s/step
Epoch 2/20
768/768 - 1399s - loss: 1.9295 - dense_4_loss: 1.1902 - dense_1_loss: 1.2389 - dense_3_loss: 1.2253 - dense_4_accuracy: 0.4391 - dense_1_accuracy: 0.4287 - dense_3_accuracy: 0.4253 - val_loss: 1.8173 - val_dense_4_loss: 1.1168 - val_dense_1_loss: 1.1798 - val_dense_3_loss: 1.1553 - val_dense_4_accuracy: 0.4909 - val_dense_1_accuracy: 0.4609 - val_dense_3_accuracy: 0.4720 - 1399s/epoch - 2s/step
Epoch 3/20
768/768 - 1398s - loss: 1.8243 - dense_4_loss: 1.1189 - dense_1_loss: 1.1890 - dense_3_loss: 1.1622 - dense_4_accuracy: 0.4871 - de

Testing time (s) = 28116.090969450073

for testing
60/60 [==============================] - 92s 2s/step - loss: 1.2273 - dense_4_loss: 0.7016 - dense_1_loss: 0.9575 - dense_3_loss: 0.7947 - dense_4_accuracy: 0.6964 - dense_1_accuracy: 0.5911 - dense_3_accuracy: 0.6578
[1.227288007736206, 0.7016499042510986, 0.9574624300003052, 0.794665515422821, 0.6963541507720947, 0.5911458134651184, 0.6578124761581421]
for training


2023-01-07 02:45:04.727000: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 4644864000 exceeds 10% of free system memory.


240/240 [==============================] - 361s 2s/step - loss: 1.2033 - dense_4_loss: 0.6826 - dense_1_loss: 0.9535 - dense_3_loss: 0.7821 - dense_4_accuracy: 0.7214 - dense_1_accuracy: 0.5987 - dense_3_accuracy: 0.6742
[1.203284502029419, 0.6826215982437134, 0.9534757733345032, 0.7820674180984497, 0.7213541865348816, 0.5986979007720947, 0.6742187738418579]
From epochs:  21  to  40
Testing started
Epoch 1/20
768/768 - 1395s - loss: 1.2383 - dense_4_loss: 0.7028 - dense_1_loss: 0.9804 - dense_3_loss: 0.8043 - dense_4_accuracy: 0.6994 - dense_1_accuracy: 0.5719 - dense_3_accuracy: 0.6486 - val_loss: 1.1852 - val_dense_4_loss: 0.6687 - val_dense_1_loss: 0.9480 - val_dense_3_loss: 0.7736 - val_dense_4_accuracy: 0.7246 - val_dense_1_accuracy: 0.5911 - val_dense_3_accuracy: 0.6686 - 1395s/epoch - 2s/step
Epoch 2/20
768/768 - 1390s - loss: 1.2208 - dense_4_loss: 0.6912 - dense_1_loss: 0.9665 - dense_3_loss: 0.7989 - dense_4_accuracy: 0.7049 - dense_1_accuracy: 0.5841 - dense_3_accuracy: 0.64

768/768 - 1415s - loss: 1.0715 - dense_4_loss: 0.5890 - dense_1_loss: 0.9070 - dense_3_loss: 0.7013 - dense_4_accuracy: 0.7572 - dense_1_accuracy: 0.6123 - dense_3_accuracy: 0.7031 - val_loss: 1.0273 - val_dense_4_loss: 0.5599 - val_dense_1_loss: 0.8830 - val_dense_3_loss: 0.6751 - val_dense_4_accuracy: 0.7708 - val_dense_1_accuracy: 0.6237 - val_dense_3_accuracy: 0.7109 - 1415s/epoch - 2s/step
Testing time (s) = 28125.237555703847

for testing
60/60 [==============================] - 92s 2s/step - loss: 1.0536 - dense_4_loss: 0.5811 - dense_1_loss: 0.8887 - dense_3_loss: 0.6864 - dense_4_accuracy: 0.7563 - dense_1_accuracy: 0.6187 - dense_3_accuracy: 0.7042
[1.0535887479782104, 0.5810701847076416, 0.8886677026748657, 0.6863946318626404, 0.7562500238418579, 0.6187499761581421, 0.7041666507720947]
for training


2023-01-07 10:41:32.814862: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 4644864000 exceeds 10% of free system memory.


240/240 [==============================] - 362s 2s/step - loss: 1.0282 - dense_4_loss: 0.5620 - dense_1_loss: 0.8826 - dense_3_loss: 0.6713 - dense_4_accuracy: 0.7715 - dense_1_accuracy: 0.6306 - dense_3_accuracy: 0.7294
[1.0282045602798462, 0.5620396733283997, 0.8826070427894592, 0.6712760329246521, 0.771484375, 0.6305989623069763, 0.7294270992279053]


In [27]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 30, 30, 168  0           []                               
                                )]                                                                
                                                                                                  
 resizing (Resizing)            (None, 224, 224, 16  0           ['input_1[0][0]']                
                                8)                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 112, 112, 64  526912      ['resizing[0][0]']               
                                )                                                             

 max_pooling2d_6 (MaxPooling2D)  (None, 13, 13, 512)  0          ['tf.concat_2[0][0]']            
                                                                                                  
 conv2d_22 (Conv2D)             (None, 13, 13, 160)  82080       ['tf.concat_2[0][0]']            
                                                                                                  
 conv2d_24 (Conv2D)             (None, 13, 13, 224)  25312       ['conv2d_23[0][0]']              
                                                                                                  
 conv2d_26 (Conv2D)             (None, 13, 13, 64)   1600        ['conv2d_25[0][0]']              
                                                                                                  
 conv2d_27 (Conv2D)             (None, 13, 13, 64)   32832       ['max_pooling2d_6[0][0]']        
                                                                                                  
 tf.concat

                                                                                                  
 conv2d_49 (Conv2D)             (None, 6, 6, 320)    51520       ['conv2d_48[0][0]']              
                                                                                                  
 conv2d_51 (Conv2D)             (None, 6, 6, 128)    4224        ['conv2d_50[0][0]']              
                                                                                                  
 conv2d_52 (Conv2D)             (None, 6, 6, 128)    106624      ['max_pooling2d_11[0][0]']       
                                                                                                  
 tf.concat_7 (TFOpLambda)       (None, 6, 6, 832)    0           ['conv2d_47[0][0]',              
                                                                  'conv2d_49[0][0]',              
                                                                  'conv2d_51[0][0]',              
          

In [30]:
for x in range(2,3):
    print("From epochs: ",20*x+1," to ",20+20*x)
    tic = start_timer()
    model.fit(x_train ,y_train ,batch_size=BATCH_SIZE ,epochs=20, verbose=2, validation_data=(x_val, y_val), shuffle=True)
    toc = end_timer()
    show_time(tic,toc)
    print("for testing")
    print(model.evaluate(test_dataset,test_dataset_label))
    print("for training")
    print(model.evaluate(train_dataset,train_dataset_label))

From epochs:  41  to  60
Testing started
Epoch 1/20
768/768 - 1420s - loss: 1.0645 - dense_4_loss: 0.5833 - dense_1_loss: 0.9033 - dense_3_loss: 0.7007 - dense_4_accuracy: 0.7565 - dense_1_accuracy: 0.6064 - dense_3_accuracy: 0.7077 - val_loss: 1.0333 - val_dense_4_loss: 0.5647 - val_dense_1_loss: 0.8824 - val_dense_3_loss: 0.6796 - val_dense_4_accuracy: 0.7643 - val_dense_1_accuracy: 0.6172 - val_dense_3_accuracy: 0.7070 - 1420s/epoch - 2s/step
Epoch 2/20
768/768 - 1406s - loss: 1.0600 - dense_4_loss: 0.5799 - dense_1_loss: 0.9018 - dense_3_loss: 0.6983 - dense_4_accuracy: 0.7573 - dense_1_accuracy: 0.6162 - dense_3_accuracy: 0.6997 - val_loss: 1.0214 - val_dense_4_loss: 0.5563 - val_dense_1_loss: 0.8786 - val_dense_3_loss: 0.6716 - val_dense_4_accuracy: 0.7624 - val_dense_1_accuracy: 0.6237 - val_dense_3_accuracy: 0.7116 - 1406s/epoch - 2s/step
Epoch 3/20
768/768 - 1412s - loss: 1.0577 - dense_4_loss: 0.5803 - dense_1_loss: 0.8982 - dense_3_loss: 0.6933 - dense_4_accuracy: 0.7578 - d

Testing time (s) = 28203.969741594978

for testing
60/60 [==============================] - 91s 2s/step - loss: 0.9777 - dense_4_loss: 0.5294 - dense_1_loss: 0.8532 - dense_3_loss: 0.6409 - dense_4_accuracy: 0.7818 - dense_1_accuracy: 0.6380 - dense_3_accuracy: 0.7234
[0.9776585102081299, 0.5294498205184937, 0.8531547784805298, 0.640874445438385, 0.7817708253860474, 0.6380208134651184, 0.723437488079071]
for training


2023-01-07 18:46:47.926402: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 4644864000 exceeds 10% of free system memory.


240/240 [==============================] - 366s 2s/step - loss: 0.9492 - dense_4_loss: 0.5086 - dense_1_loss: 0.8458 - dense_3_loss: 0.6231 - dense_4_accuracy: 0.7986 - dense_1_accuracy: 0.6501 - dense_3_accuracy: 0.7513
[0.9492347240447998, 0.5085824728012085, 0.8457593321800232, 0.6230818033218384, 0.7985677123069763, 0.6501302123069763, 0.7513020634651184]
